### Load QM9 model and predict

In [1]:
from megnet.models import MEGNetModel
import numpy as np
from operator import itemgetter
import json

def get_graph_from_doc(doc):
    """
    Convert a json document into a megnet graph
    """
    atom = [i['atomic_num'] for i in doc['atoms']]

    index1_temp = [i['a_idx'] for i in doc['atom_pairs']]
    index2_temp = [i['b_idx'] for i in doc['atom_pairs']]
    bond_temp = [i['spatial_distance'] for i in doc['atom_pairs']]

    index1 = index1_temp + index2_temp
    index2 = index2_temp + index1_temp
    bond = bond_temp + bond_temp
    sort_key = np.argsort(index1)
    it = itemgetter(*sort_key)

    index1 = it(index1)
    index2 = it(index2)
    bond = it(bond)
    graph = {'atom': atom, 'bond': bond, 'index1': index1, 'index2': index2, 'state': [[0, 0]]}
    return graph
    
# load an example qm9 document
with open('../megnet/data/tests/qm9/000001.json', 'r') as f:
    doc = json.load(f)
# convert to a graph
graph = get_graph_from_doc(doc)

2024-04-22 06:21:52.248205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:21:52.248240: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# all target names
names = ['mu', 'alpha', 'HOMO', 'LUMO', 'gap', 'R2', 'ZPVE', 'U0', 'U', 'H', 'G', 'Cv', 'omega1']


y_pred = []
y_true = []

print('*** Result Comparisons ***')
print('Target\tMEGNet\tQM9')

for i in names:
    model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/' + i+'.hdf5')
    pred = model.predict_graph(graph)
    y_pred.append(pred)
    y_true.append(doc['mol_info'][i])
    print('%s\t%.3f\t%.3f' %(i, y_pred[-1], float(y_true[-1])))


*** Result Comparisons ***
Target	MEGNet	QM9


/home/shibuM/anaconda3/envs/megnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-22 06:22:02.222127: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:22:02.222204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:22:02.222252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcu

mu	-0.008	0.000
alpha	13.127	13.210
HOMO	-10.557	-10.550
LUMO	3.241	3.186
gap	13.622	13.736
R2	35.975	35.364
ZPVE	1.215	1.218
U0	-17.166	-17.172
U	-17.353	-17.286
H	-17.420	-17.389
G	-16.107	-16.152
Cv	6.427	6.469
omega1	3151.625	3151.708


### Predict from SMILES

In [3]:
from megnet.utils.molecule import get_pmg_mol_from_smiles

MODEL_NAME = 'HOMO'

model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/%s.hdf5' % MODEL_NAME)

In [4]:
# The smiles of qm9:000001 is just C
mol1 = get_pmg_mol_from_smiles('C')
model.predict_structure(mol1)

array([-10.557696], dtype=float32)

The result matches with previous results when we compute it from pre-computed graph